In [1]:
import torch
import torchaudio
from nnAudio import features
from scipy.io import wavfile
from nnAudio import features
import torch

import openvino as ov

from torch_audioset.yamnet.model import yamnet as torch_yamnet
from torch_audioset.data.torch_input_processing import WaveformToInput as TorchTransform
import onnxruntime

In [2]:
wav_file = '/data/audio/ITW-Music/snr10-inthewild-16khz-inthewild_10002.wav'
waveform, sample_rate = torchaudio.load(wav_file, normalize=True)
# transform = torchaudio.transforms.Spectrogram(n_fft=512)
# spectrogram = transform(waveform)

## Export frontend (mel-spectrogram)

In [ ]:
import custom_yamnet_model

frontend_model = custom_yamnet_model.ModelFrontend()
specgram = frontend_model(waveform)
frontend_model.eval()
torch.onnx.export(frontend_model,               # model being run
    waveform,                         # model input (or a tuple for multiple inputs)
    'yamnet_frontend.onnx',            # where to save the model (can be a file or file-like object)
    verbose=True,
    input_names = ['input'],
    dynamic_axes={
        'input' : {0 : 'chunk_length'},
        }
    )

STFT kernels created, time used = 0.0055 seconds


## Export only YamNet (no spectrogram)

In [ ]:
pt_model = torch_yamnet(pretrained=False)
pt_model.load_state_dict(torch.load('./yamnet.pth'))

patches, spectrogram = TorchTransform().wavform_to_log_mel(waveform, 16000)

In [ ]:
with torch.no_grad():
    pt_model.eval()
    torch.onnx.export(pt_model,               # model being run
        patches,                         # model input (or a tuple for multiple inputs)
        'yamnet.onnx',            # where to save the model (can be a file or file-like object)
        verbose=True)

## Export E2E (spectrogram + YamNet)

In [ ]:

tt_model = TorchTransform()

patches, spectrogram = tt_model.wavform_to_log_mel(waveform, 16000)

patches2 = tt_model(waveform, sample_rate)

spectrogram_direct = tt_model.mel_trans_ope.spectrogram(waveform)
spectrogram_direct = spectrogram_direct ** 0.5
mel_specgram_direct= tt_model.mel_trans_ope.mel_scale(spectrogram_direct)
mel_specgram_direct = torch.log(mel_specgram_direct + 0.001)
mel_specgram_direct = mel_specgram_direct.squeeze(dim=0).T 


In [ ]:
import custom_yamnet_model as Model

model = Model()    

with torch.no_grad():
    model.eval()
    y, spectrogram = model(waveform) # automatically convert waveforms into spectrograms

    torch.onnx.export(model,               # model being run
        waveform,                         # model input (or a tuple for multiple inputs)
        'yamnet_e2e.onnx',            # where to save the model (can be a file or file-like object)
        verbose=True)

    # print(f'STFT all close: {torch.allclose(x_stft, x_stft_mel)}')

    # y = y.squeeze(dim=0).T 
    # y = torch.log(y + 0.001)
    print(f'min val: {torch.min(y)}, max val: {torch.max(y)}, mean val: {torch.mean(y)}, std val: {torch.std(y)}')
    torch.allclose(patches, y, atol=1e-5)


pt_model = torch_yamnet(pretrained=False)
# Manually download the `yamnet.pth` file.
pt_model.load_state_dict(torch.load('./yamnet.pth'))

patches, spectrogram = TorchTransform().wavform_to_log_mel(waveform, 16000)


with torch.no_grad():
    pt_model.eval()
    # x = torch.from_numpy(patches)
    # x = x.unsqueeze(1)  # [5, 96, 64] -> [5, 1, 96, 64]
    x = y # patches
    pt_pred = pt_model(x, to_prob=True)
    # pt_pred = pt_pred.numpy()
    
    pt_pred2 = pt_model(patches, to_prob=True)
    


In [ ]:
model = TorchTransform()
x = model(waveform, sample_rate)
torch.onnx.export(model,               # model being run
    (waveform, x),                         # model input (or a tuple for multiple inputs)
    'torch_transform.onnx',            # where to save the model (can be a file or file-like object)
    verbose=True)

In [ ]:
# using nnAudio
sr = 16000
waveforms = torch.randn(1,sr)

spec_layer = features.STFT(n_fft=512, win_length=0.025*sr, freq_bins=None, hop_length=0.010*sr,
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect',
                              fmin=50,fmax=11025, sr=sr) # Initializing the model
spectrogram_nnaudio = spec_layer(waveforms)

In [ ]:
spec_layer.eval()
torch.onnx.export(spec_layer,               # model being run
    waveform,                         # model input (or a tuple for multiple inputs)
    'specgram_nnaudio.onnx',            # where to save the model (can be a file or file-like object)
    verbose=True)

In [ ]:
transform.eval()
torch.onnx.export(transform,               # model being run
    waveform,                         # model input (or a tuple for multiple inputs)
    'specgram.onnx',            # where to save the model (can be a file or file-like object)
    verbose=True)